# Ejercicio 2 - Sumar matrices con kernel 2D

Partiendo del código suministrado:

* Completar los fragmentos indicados con `TODO` 



Sugerencias

1. Completar el kernel normalmente

2. Implementar dos versiones con acceso recorriendo filas y columnas y usar la variable booleana coalesced para poder elegir entre las dos

3. Comparar los tiempos de ejecución

4. El acceso es coalescente ¿por filas o por columnas?


---

In [1]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [5]:
import numpy as np
from numba import cuda

n = 4096

@cuda.jit
def matrix_add(a, b, out, coalesced):
    x, y = cuda.grid(2)

    if coalesced: # acceso COALESCENTE
        out[y][x] = a[y][x] + b[y][x]

    else: # acceso NO COALESCENTE
        out[x][y] = a[x][y] + b[x][y]

# PARÁMETROS
threads_per_block = (32, 32)  # 2D block
blocks = (128, 128) # 2D grid

# VARIABLES en CPU
h_a = np.arange(n*n).reshape(n,n).astype(np.float32)
h_b = h_a.copy().astype(np.float32)

# VARIABLES en GPU
d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_out_no_coalesced = cuda.device_array_like(h_a)
d_out_coalesced = cuda.device_array_like(h_a)

# LANZA KERNEL - acceso no coalescente
matrix_add[blocks, threads_per_block](d_a, d_b, d_out_no_coalesced, False)
h_out_no_coalesced = d_out_no_coalesced.copy_to_host() # resultado a CPU

# LANZA KERNEL - acceso coalescente
matrix_add[blocks, threads_per_block](d_a, d_b, d_out_coalesced, True)
h_out_coalesced = d_out_coalesced.copy_to_host() # resultado a CPU

# COMPROBACIÓN
truth = h_a+h_b
np.testing.assert_equal(h_out_no_coalesced, truth)
np.testing.assert_equal(h_out_coalesced, truth)

# MEDICIÓN - acceso coalescente
# cpu
print("\nTiempo en CPU:")
%timeit h_a+h_b

# gpu no coalescente
print("\nTiempo en GPU - acceso no coalescente:")
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out_no_coalesced, False); cuda.synchronize()

# gpu coalescente
print("\nTiempo en GPU - acceso coalescente:")
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out_coalesced, True); cuda.synchronize()


Tiempo en CPU:
13 ms ± 374 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Tiempo en GPU - acceso no coalescente:
2.55 ms ± 7.72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Tiempo en GPU - acceso coalescente:
863 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
